In [2]:
import requests
import pandas as pd

In [4]:
BASE = "https://data.austintexas.gov/resource/3syk-w9eu.json"
params = {
    "$select": "permit_number, issue_date, status_current",
    "$limit": 2000,
    "$offset": 0,
    "$$app_token": "PASTE_YOUR_APP_TOKEN_HERE",  # optional but recommended
}

df = pd.read_json(BASE, params=params)
df.head()

TypeError: read_json() got an unexpected keyword argument 'params'

In [6]:
import duckdb

In [8]:
CSV = "C:/Users/siddh/Downloads/Issued_Construction_Permits_20251212.csv"  

# Peek at rows
duckdb.query(f"""
    SELECT *
    FROM read_csv_auto('{CSV}')
    LIMIT 20
""").df()

,Permit Type Desc,Permit Num,Permit Class Mapped,Permit Type,Permit Class,Work Class,Condominium,Project Name,Description,TCAD ID,...,Contractor Zip,Applicant Full Name,Applicant Organization,Applicant Phone,Applicant Address 1,Applicant Address 2,Applicant City,Applicant Zip,Certificate Of Occupancy,Total Lot SQFT
0,Electrical Permit,2025-156626 EP,Residential,EP,Residential,Homebuilder Loop,<NA>,5505 WILLIAM HOLLAND AVE,Tpole,0229033301,...,78724,None,None,None,None,None,None,None,<NA>,None
1,Electrical Permit,2025-037178 EP,Residential,EP,R- 101 Single Family Houses,New,False,8618 DONGAN DR,New 2 Story SFR with 4 beds 3.5 baths with att...,0340132017,...,78634-____,None,None,None,None,None,None,None,False,None
2,Electrical Permit,2025-156220 EP,Residential,EP,Residential,Repair,False,12610 VELARDE CV,Permit required for Outage and Service Upgrade,R066275,...,78258,None,None,None,None,None,None,None,False,None
3,Plumbing Permit,2025-156491 PP,Residential,PP,Residential,New,False,14124 EDNA MAXINE WALK,New Construction in the ETJ SFR,0331410510,...,78634,None,None,None,None,None,None,None,False,None
4,Plumbing Permit,2025-156399 PP,Commercial,PP,Commercial,Irrigation,<NA>,4908 LOTT AVE,New commercial irrigation\nREF: 2025-093375W,None,...,None,None,None,None,None,None,None,None,<NA>,None
5,Building Permit,2025-155472 BP,Commercial,BP,C-1000 Commercial Remodel,Remodel,False,4140 GOVERNORS ROW,Reroof,0412000401,...,78724,Molly Johnston,Texas Fifth Wall Roofing Systems,2108673598,None,3300 Duke Rd,Austin,78724,False,None
6,Driveway / Sidewalks,2025-045619 DS,Residential,DS,Res. Driveway,New,<NA>,3405 GONZALES ST,Construct new driveway approach of 10 FT,0202140804,...,None,None,None,None,None,None,None,None,<NA>,None
7,Building Permit,2025-151746 BP,Commercial,BP,C-1000 Commercial Remodel,Remodel,False,6805 WOOD HOLLOW DR BLDG 3,Install 22 Steel Piers 6 Helical Piers 19 LF o...,0136010211,...,75050-____,Scott Bedford,PermaPier Foundation Repair of Texas,9728421993,None,1121 Fountain Pkwy,Grand Prairie,75050-____,False,None
8,Electrical Permit,2025-139720 EP,Commercial,EP,Sign Permit,Wall,<NA>,3500 E PARMER LN BLDG 8,New Sign Installation,0247380308,...,None,None,None,None,None,None,None,None,<NA>,None
9,Electrical Permit,2025-139747 EP,Commercial,EP,Sign Permit,Wall,<NA>,3500 E PARMER LN BLDG 8,New Sign Installation,0247380308,...,None,None,None,None,None,None,None,None,<NA>,None


## Get a mix across years/types/status to see the variety.

In [9]:
q = f"""
SELECT
  permit_number,
  issue_date,
  calendar_year_issued,
  permit_type_desc,
  work_class,
  permit_class,
  description,
  permit_location,
  original_zip,
  status_current,
  total_job_valuation
FROM read_csv_auto('{CSV}')
WHERE description IS NOT NULL
ORDER BY random()
LIMIT 50
"""
review = duckdb.query(q).df()
review


BinderException: Binder Error: Referenced column "permit_number" not found in FROM clause!
Candidate bindings: "Permit Num", "Permit Type", "Description", "Permit Type Desc", "Permit Class"

In [10]:
duckdb.query(f"""
DESCRIBE SELECT * FROM read_csv_auto('{CSV}')
""").df()

,column_name,column_type,null,key,default,extra
0,Permit Type Desc,VARCHAR,YES,None,None,None
1,Permit Num,VARCHAR,YES,None,None,None
2,Permit Class Mapped,VARCHAR,YES,None,None,None
3,Permit Type,VARCHAR,YES,None,None,None
4,Permit Class,VARCHAR,YES,None,None,None
...,...,...,...,...,...,...
63,Applicant Address 2,VARCHAR,YES,None,None,None
64,Applicant City,VARCHAR,YES,None,None,None
65,Applicant Zip,VARCHAR,YES,None,None,None
66,Certificate Of Occupancy,BOOLEAN,YES,None,None,None


In [11]:
duckdb.query(f"""
CREATE OR REPLACE VIEW permits AS
SELECT
  "Permit Type Desc"           AS permit_type_desc,
  "Permit Num"                 AS permit_number,
  "Permit Class Mapped"        AS permit_class_mapped,
  "Permit Type"                AS permittype,
  "Permit Class"               AS permit_class,
  "Work Class"                 AS work_class,
  "Condominium"                AS condominium,
  "Permit Location"            AS permit_location,
  "Description"                AS description,
  "TCAD ID"                    AS tcad_id,
  "Legal Description"          AS legal_description,
  "AppliedDate"                AS applieddate,
  "Issue Date"                 AS issue_date,
  "Day Issued"                 AS day_issued,
  "Calendar Year Issued"       AS calendar_year_issued,
  "Fiscal Year Issued"         AS fiscal_year_issued,
  "Issued in Last 30 Days"     AS issued_in_last_30_days,
  "Issue Method"               AS issue_method,
  "Status Current"             AS status_current,
  "StatusDate"                 AS statusdate,
  "ExpiresDate"                AS expiresdate,
  "Completed Date"             AS completed_date,
  "Total Existing Bldg SQFT"   AS total_existing_bldg_sqft,
  "Remodel Repair SQFT"        AS remodel_repair_sqft,
  "Total New Add SQFT"         AS total_new_add_sqft,
  "Total Job Valuation"        AS total_job_valuation,
  "Original Address1"          AS original_address1,
  "Original City"              AS original_city,
  "Original State"             AS original_state,
  "Original Zip"               AS original_zip,
  "Council District"           AS council_district,
  "Jurisdiction"               AS jurisdiction,
  "Latitude"                   AS latitude,
  "Longitude"                  AS longitude
FROM read_csv_auto('{CSV}')
""")

BinderException: Binder Error: Referenced column "Permit Location" not found in FROM clause!
Candidate bindings: "Permit Class", "Permit Num", "Permit Type", "Permit Type Desc", "Electrical Valuation"

LINE 11:   "Permit Location"            AS permit_location,
           ^

In [12]:
duckdb.query(f"""
CREATE OR REPLACE VIEW permits AS
SELECT
  "Permit Num"        AS permit_number,
  "Permit Type Desc"  AS permit_type_desc,
  "Permit Type"       AS permittype,
  "Permit Class"      AS permit_class,
  "Work Class"        AS work_class,
  "Description"       AS description
FROM read_csv_auto('{CSV}')
""")

In [13]:
duckdb.query("""
SELECT *
FROM permits
WHERE description IS NOT NULL
USING SAMPLE 20
""").df()


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,permit_number,permit_type_desc,permittype,permit_class,work_class,description
0,2025-042275 PP,Plumbing Permit,PP,R- 101 Single Family Houses,New,Highland 3313 B Right New - 1 Story SFR CONDOM...
1,2016-056792 PP,Plumbing Permit,PP,Commercial,Fireline,Install outside underground fireline only for ...
2,2025-063966 BP,Building Permit,BP,C-1000 Commercial Remodel,Remodel,ePlan: Commercial Expedited Review - Interior ...
3,2025-003219 MP,Mechanical Permit,MP,R- 101 Single Family Houses,New,PB: New construction of 2 story SFR 4 Bed 3 Ba...
4,2025-145133 MP,Mechanical Permit,MP,Residential,Change Out,Replacement of complete existing central heat ...
5,2024-162558 BP,Building Permit,BP,R- 329 Res Structures Other Than Bldg,New,ePlan: Residential Expedited Review - New Pool...
6,2024-159893 EP,Electrical Permit,EP,Residential,Homebuilder Loop,Build 200 aMP TEMP SERVICE
7,2025-030007 EP,Electrical Permit,EP,C-1000 Commercial Remodel,Remodel,THIS PROJECT CONSISTS OF RENOVATION ON THE GRO...
8,2024-095378 PP,Plumbing Permit,PP,R- 434 Addition & Alterations,Addition and Remodel,Interior Remodel and Garage conversion\n\nInte...
9,2025-029144 BP,Building Permit,BP,R- 102 Secondary Apartment,New,New 1-story 1 bedroom 1 bathroom secondary dwe...


In [14]:
desc = duckdb.query(f"DESCRIBE SELECT * FROM read_csv_auto('{CSV}')").df()
desc[desc["column_name"].str.contains("Permit", case=False, na=False)].head(30)


,column_name,column_type,null,key,default,extra
0,Permit Type Desc,VARCHAR,YES,None,None,None
1,Permit Num,VARCHAR,YES,None,None,None
2,Permit Class Mapped,VARCHAR,YES,None,None,None
3,Permit Type,VARCHAR,YES,None,None,None
4,Permit Class,VARCHAR,YES,None,None,None
47,Master Permit Num,BIGINT,YES,None,None,None


In [15]:
desc = duckdb.query(f"DESCRIBE SELECT * FROM read_csv_auto('{CSV}')").df()
cols = desc["column_name"].tolist()

def find_cols(*needles):
    needles = [n.lower() for n in needles]
    return [c for c in cols if any(n in c.lower() for n in needles)]

find_cols("issue", "date", "status", "address", "city", "state", "zip", "lat", "lon", "location", "valuation", "work", "class", "description")


['Permit Class Mapped',
 'Permit Class',
 'Work Class',
 'Description',
 'Property Legal Description',
 'Applied Date',
 'Issued Date',
 'Day Issued',
 'Calendar Year Issued',
 'Fiscal Year Issued',
 'Issued In Last 30 Days',
 'Status Current',
 'Status Date',
 'Expires Date',
 'Completed Date',
 'Total Valuation Remodel',
 'Total Job Valuation',
 'Building Valuation',
 'Building Valuation Remodel',
 'Electrical Valuation',
 'Electrical Valuation Remodel',
 'Mechanical Valuation',
 'Mechanical Valuation Remodel',
 'Plumbing Valuation',
 'Plumbing Valuation Remodel',
 'MedGas Valuation',
 'MedGas Valuation Remodel',
 'Original Address 1',
 'Original City',
 'Original State',
 'Original Zip',
 'Latitude',
 'Longitude',
 'Location',
 'Contractor Address 1',
 'Contractor Address 2',
 'Contractor City',
 'Contractor Zip',
 'Applicant Address 1',
 'Applicant Address 2',
 'Applicant City',
 'Applicant Zip']

In [16]:
duckdb.query(f"""
CREATE OR REPLACE VIEW permits AS
SELECT
  "Permit Num"               AS permit_number,
  "Permit Type Desc"         AS permit_type_desc,
  "Permit Type"              AS permittype,
  "Permit Class"             AS permit_class,
  "Permit Class Mapped"      AS permit_class_mapped,
  "Work Class"               AS work_class,
  "Description"              AS description,

  "Applied Date"             AS applied_date,
  "Issued Date"              AS issued_date,
  "Day Issued"               AS day_issued,
  "Calendar Year Issued"     AS calendar_year_issued,
  "Fiscal Year Issued"       AS fiscal_year_issued,
  "Issued In Last 30 Days"   AS issued_in_last_30_days,

  "Status Current"           AS status_current,
  "Status Date"              AS status_date,
  "Expires Date"             AS expires_date,
  "Completed Date"           AS completed_date,

  TRY_CAST("Total Job Valuation" AS DOUBLE)      AS total_job_valuation,
  TRY_CAST("Total Valuation Remodel" AS DOUBLE)  AS total_valuation_remodel,
  TRY_CAST("Building Valuation" AS DOUBLE)       AS building_valuation,
  TRY_CAST("Electrical Valuation" AS DOUBLE)     AS electrical_valuation,
  TRY_CAST("Mechanical Valuation" AS DOUBLE)     AS mechanical_valuation,
  TRY_CAST("Plumbing Valuation" AS DOUBLE)       AS plumbing_valuation,

  "Original Address 1"       AS original_address1,
  "Original City"            AS original_city,
  "Original State"           AS original_state,
  "Original Zip"             AS original_zip,
  TRY_CAST("Latitude" AS DOUBLE)                 AS latitude,
  TRY_CAST("Longitude" AS DOUBLE)                AS longitude,
  "Location"                 AS location,

  "Contractor City"          AS contractor_city,
  "Contractor Zip"           AS contractor_zip,
  "Applicant City"           AS applicant_city,
  "Applicant Zip"            AS applicant_zip

FROM read_csv_auto('{CSV}')
""")


In [17]:
duckdb.query("""
SELECT
  permit_number, issued_date, status_current, original_zip, total_job_valuation, description
FROM permits
WHERE description IS NOT NULL
USING SAMPLE 20
""").df()


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,permit_number,issued_date,status_current,original_zip,total_job_valuation,description
0,2017-090174 EP,2025-08-18,Final,78704,NaN,new detached carport A
1,2023-038921 BP,2024-02-29,Final,78744,NaN,Total demo of a detached garage 484 sq ft\n***...
2,2025-059237 BP,2025-05-22,Active,78704,NaN,New in-ground pool (513 sqft)
3,2024-163193 PP,2024-12-20,Final,78747,NaN,Sheldon S403 B Right New - 2 Story SFR w/ 3 be...
4,2022-108384 MP,2022-08-16,Final,78701,NaN,Interior Finish out for Restaurant
5,2024-135874 EP,2024-10-10,Final,78703,NaN,REQUEST TO DISCONNECT EXISTING 200 AMP METER C...
6,2024-070491 MP,2024-06-06,Expired,78721,NaN,Replacement of complete existing central heat ...
7,2025-064722 BP,2025-05-28,Active,78704,NaN,INTERIOR REMODEL TO AN EXISTING 1977 SF HOME. ...
8,2024-152535 EP,2024-11-20,Final,78748,NaN,Installation of a 226.98 kW roof mounted solar...
9,2025-083125 EP,2025-07-07,Final,78704,NaN,T Pole


In [18]:
duckdb.query(f"""
CREATE OR REPLACE VIEW permits AS
SELECT
  "Permit Num"               AS permit_number,
  "Permit Type Desc"         AS permit_type_desc,
  "Permit Type"              AS permittype,
  "Permit Class"             AS permit_class,
  "Permit Class Mapped"      AS permit_class_mapped,
  "Work Class"               AS work_class,
  "Description"              AS description,

  "Applied Date"             AS applied_date,
  "Issued Date"              AS issued_date,

  "Status Current"           AS status_current,
  "Original Zip"             AS original_zip,

  TRY_CAST(regexp_replace(COALESCE("Total Job Valuation", ''), '[^0-9.]', '', 'g') AS DOUBLE)
    AS total_job_valuation

FROM read_csv_auto('{CSV}')
""")


In [19]:
duckdb.query("""
SELECT "Total Job Valuation", total_job_valuation
FROM permits
WHERE "Total Job Valuation" IS NOT NULL
USING SAMPLE 20
""").df()


BinderException: Binder Error: Referenced column "Total Job Valuation" not found in FROM clause!
Candidate bindings: "total_job_valuation", "original_zip"

In [20]:
duckdb.query(f"""
SELECT "Total Job Valuation"
FROM read_csv_auto('{CSV}')
WHERE "Total Job Valuation" IS NOT NULL
LIMIT 20
""").df()

,Total Job Valuation
0,$1.00
1,$0.00
2,$1.00
3,$1.00
4,$0.00
5,"$9,000.00"
6,$1.00
7,$1.00
8,$0.00
9,$0.00


In [21]:
duckdb.query(f"""
CREATE OR REPLACE VIEW permits AS
SELECT
  "Permit Num"               AS permit_number,
  "Permit Type Desc"         AS permit_type_desc,
  "Permit Type"              AS permittype,
  "Permit Class"             AS permit_class,
  "Permit Class Mapped"      AS permit_class_mapped,
  "Work Class"               AS work_class,
  "Description"              AS description,

  "Issued Date"              AS issued_date,
  "Status Current"           AS status_current,
  "Original Zip"             AS original_zip,

  TRY_CAST(
    NULLIF(regexp_replace("Total Job Valuation", '[^0-9.]', '', 'g'), '')
    AS DOUBLE
  ) AS total_job_valuation

FROM read_csv_auto('{CSV}')
""")


In [22]:
duckdb.query("""
CREATE OR REPLACE VIEW permits_clean AS
SELECT
  *,
  CASE
    WHEN total_job_valuation IN (0, 1) THEN NULL
    ELSE total_job_valuation
  END AS total_job_valuation_clean
FROM permits
""")


In [23]:
duckdb.query("""
SELECT
  COUNT(*) AS total_rows,
  SUM(CASE WHEN total_job_valuation_clean IS NULL THEN 1 ELSE 0 END) AS null_or_placeholder,
  SUM(CASE WHEN total_job_valuation_clean IS NOT NULL THEN 1 ELSE 0 END) AS usable
FROM permits_clean
""").df()


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,total_rows,null_or_placeholder,usable
0,2333734,1200529.0,1133205.0


In [24]:
phrases = duckdb.query("""
WITH base AS (
  SELECT
    lower(regexp_replace(description, '[^a-z0-9 ]', ' ', 'g')) AS d
  FROM permits_clean
  WHERE description IS NOT NULL
  USING SAMPLE 300000
),
tok AS (
  SELECT string_split(d, ' ') AS w
  FROM base
),
bigrams AS (
  SELECT lower(w[i] || ' ' || w[i+1]) AS phrase
  FROM tok, range(1, array_length(w)-1) r(i)
),
filtered AS (
  SELECT phrase, COUNT(*) AS freq
  FROM bigrams
  WHERE length(phrase) >= 6
    AND phrase NOT LIKE '%permit%'
    AND phrase NOT LIKE '%review%'
    AND phrase NOT LIKE '%expedited%'
    AND phrase NOT LIKE '%eplan%'
    AND phrase NOT LIKE '%residential expedited%'
  GROUP BY 1
)
SELECT phrase, freq
FROM filtered
WHERE freq >= 300
ORDER BY freq DESC
LIMIT 200
""").df()

phrases.head(30)


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,phrase,freq
0,arage,26352
1,emodel,25110
2,arage,24762
3,esidence,22063
4,esidence,18845
5,single family,17541
6,nterior,17391
7,emodel,17352
8,to existing,16922
9,attached garage,16752


In [25]:
phrases["head"] = phrases["phrase"].str.split().str[-1]
grouped = (phrases.groupby("head", as_index=False)
           .agg(phrase_count=("phrase","count"),
                total_freq=("freq","sum"),
                top_phrases=("phrase", lambda s: list(s.head(8)))))
grouped = grouped.sort_values(["phrase_count","total_freq"], ascending=False)
grouped.head(30)


,head,phrase_count,total_freq,top_phrases
39,existing,7,41225,"[to existing, existing , complete existing, of..."
63,of,7,21105,"[eplacement of, onstruction of, construction o..."
92,to,5,16472,"[remodel to, emodel to, system to, heater to, ..."
6,and,4,24075,"[porch and, heat and, sidewalk and, garage and]"
76,porch,4,23679,"[covered porch, entry porch, porch , front porch]"
73,patio,4,20465,"[and patio, patio , covered patio, rear patio]"
57,new,4,9126,"[onstruct new, of new, nstall new, for new]"
89,story,3,31201,"[2 story, 1 story, story ]"
82,residence,3,23019,"[family residence, existing residence, residen..."
43,garage,3,22562,"[attached garage, car garage, garage ]"


In [26]:
phrases = duckdb.query("""
WITH base AS (
  SELECT
    lower(trim(regexp_replace(description, '[^a-z0-9 ]', ' ', 'g'))) AS d
  FROM permits_clean
  WHERE description IS NOT NULL
  USING SAMPLE 300000
),
norm AS (
  SELECT regexp_replace(d, '\\s+', ' ', 'g') AS d
  FROM base
),
tok AS (
  SELECT list_filter(string_split(d, ' '), x -> length(x) > 0) AS w
  FROM norm
),
bigrams AS (
  SELECT w[i] || ' ' || w[i+1] AS phrase
  FROM tok, range(1, array_length(w)) r(i)
  WHERE i < array_length(w)
),
filtered AS (
  SELECT phrase, COUNT(*) AS freq
  FROM bigrams
  WHERE length(phrase) >= 6
    AND phrase NOT LIKE '%permit%'
    AND phrase NOT LIKE '%review%'
    AND phrase NOT LIKE '%expedited%'
    AND phrase NOT LIKE '%eplan%'
  GROUP BY 1
)
SELECT phrase, freq
FROM filtered
WHERE freq >= 300
ORDER BY freq DESC
LIMIT 200
""").df()

phrases.head(30)


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,phrase,freq
0,tt arage,21970
1,single family,17743
2,attached garage,17378
3,to existing,16995
4,2 story,16318
5,garage covered,15291
6,family residence,14719
7,story single,13193
8,porch and,13130
9,try rm,11294


In [27]:
duckdb.query("""
SELECT
  SUM(CASE WHEN strpos(description, chr(160)) > 0 THEN 1 ELSE 0 END) AS has_nbsp,
  SUM(CASE WHEN strpos(description, chr(8203)) > 0 THEN 1 ELSE 0 END) AS has_zwsp
FROM permits_clean
WHERE description IS NOT NULL
""").df()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,has_nbsp,has_zwsp
0,137.0,0.0


In [28]:
phrases = duckdb.query("""
WITH base AS (
  SELECT
    lower(trim(
      regexp_replace(
        replace(replace(description, chr(160), ''), chr(8203), ''),
        '[^a-z0-9 ]', ' ', 'g'
      )
    )) AS d
  FROM permits_clean
  WHERE description IS NOT NULL
  USING SAMPLE 300000
),
norm AS (
  SELECT regexp_replace(d, '\\s+', ' ', 'g') AS d
  FROM base
),
tok AS (
  SELECT list_filter(string_split(d, ' '), x -> length(x) > 0) AS w
  FROM norm
),
bigrams AS (
  SELECT w[i] || ' ' || w[i+1] AS phrase
  FROM tok, range(1, array_length(w)) r(i)
  WHERE i < array_length(w)
),
filtered AS (
  SELECT phrase, COUNT(*) AS freq
  FROM bigrams
  WHERE length(phrase) >= 6
    AND phrase NOT LIKE '%permit%'
    AND phrase NOT LIKE '%review%'
    AND phrase NOT LIKE '%expedited%'
    AND phrase NOT LIKE '%eplan%'
  GROUP BY 1
)
SELECT phrase, freq
FROM filtered
WHERE freq >= 300
ORDER BY freq DESC
LIMIT 200
""").df()

phrases.head(30)


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,phrase,freq
0,tt arage,21496
1,single family,17398
2,to existing,17260
3,attached garage,16925
4,2 story,16228
5,garage covered,14990
6,family residence,14515
7,story single,12939
8,porch and,12854
9,try rm,11421


In [29]:
phrases = duckdb.query("""
WITH base AS (
  SELECT
    lower(trim(
      regexp_replace(
        replace(
          replace(
            replace(
              replace(description, chr(160), ''),   -- NBSP
            chr(8203), ''),                          -- ZWSP
          chr(65279), ''),                           -- BOM
        chr(8288), ''),                              -- word joiner
        '[^a-z0-9 ]', ' ', 'g'
      )
    )) AS d
  FROM permits_clean
  WHERE description IS NOT NULL
  USING SAMPLE 300000
),
norm AS (
  SELECT regexp_replace(d, '\\s+', ' ', 'g') AS d
  FROM base
),
tok AS (
  SELECT list_filter(string_split(d, ' '), x -> length(x) > 0) AS w
  FROM norm
),
bigrams AS (
  SELECT w[i] || ' ' || w[i+1] AS phrase
  FROM tok, range(1, array_length(w)) r(i)
  WHERE i < array_length(w)
),
filtered AS (
  SELECT phrase, COUNT(*) AS freq
  FROM bigrams
  WHERE length(phrase) >= 6
  GROUP BY 1
)
SELECT phrase, freq
FROM filtered
WHERE freq >= 300
ORDER BY freq DESC
LIMIT 200
""").df()

phrases.head(30)

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,phrase,freq
0,tt arage,21837
1,single family,17565
2,attached garage,17128
3,to existing,16915
4,2 story,16299
5,garage covered,15242
6,family residence,14589
7,story single,13082
8,porch and,12939
9,try rm,11383


In [30]:
phrases = duckdb.query("""
WITH base AS (
  SELECT
    lower(description) AS d
  FROM permits_clean
  WHERE description IS NOT NULL
  USING SAMPLE 300000
),
clean1 AS (
  SELECT
    -- remove common invisible chars + ASCII control chars
    regexp_replace(
      replace(replace(replace(replace(d, chr(160), ''), chr(8203), ''), chr(65279), ''), chr(8288), ''),
      '[\\x00-\\x1F\\x7F]', ' ', 'g'
    ) AS d
  FROM base
),
clean2 AS (
  SELECT
    -- keep only a-z/0-9/spaces; everything else -> space
    trim(regexp_replace(d, '[^a-z0-9 ]', ' ', 'g')) AS d
  FROM clean1
),
clean3 AS (
  SELECT
    -- collapse multi-spaces
    regexp_replace(d, '\\s+', ' ', 'g') AS d
  FROM clean2
),
stitched AS (
  SELECT
    -- stitch split words: "g arage" -> "garage", "i nterior" -> "interior"
    regexp_replace(d, '\\b([a-z])\\s+([a-z]{2,})\\b', '\\1\\2', 'g') AS d
  FROM clean3
),
tok AS (
  SELECT list_filter(string_split(d, ' '), x -> length(x) > 0) AS w
  FROM stitched
),
bigrams AS (
  SELECT w[i] || ' ' || w[i+1] AS phrase
  FROM tok, range(1, array_length(w)) r(i)
  WHERE i < array_length(w)
),
filtered AS (
  SELECT phrase, COUNT(*) AS freq
  FROM bigrams
  WHERE length(phrase) >= 6
    AND phrase NOT LIKE '%permit%'
    AND phrase NOT LIKE '%review%'
    AND phrase NOT LIKE '%expedited%'
    AND phrase NOT LIKE '%eplan%'
  GROUP BY 1
)
SELECT phrase, freq
FROM filtered
WHERE freq >= 300
ORDER BY freq DESC
LIMIT 200
""").df()

phrases.head(30)


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,phrase,freq
0,to existing,23107
1,2 story,22291
2,single family,21600
3,garage covered,21443
4,attached garage,18223
5,watt garage,16904
6,interior remodel,16285
7,family residence,15979
8,story single,15853
9,covered porch,15304


In [31]:
duckdb.query("""
WITH base AS (
  SELECT lower(description) AS raw
  FROM permits_clean
  WHERE description IS NOT NULL
  USING SAMPLE 20
),
clean AS (
  SELECT
    raw,
    regexp_replace(
      regexp_replace(
        regexp_replace(
          regexp_replace(
            replace(replace(replace(replace(raw, chr(160), ''), chr(8203), ''), chr(65279), ''), chr(8288), ''),
            '[\\x00-\\x1F\\x7F]', ' ', 'g'
          ),
          '[^a-z0-9 ]', ' ', 'g'
        ),
        '\\s+', ' ', 'g'
      ),
      '\\b([a-z])\\s+([a-z]{2,})\\b', '\\1\\2', 'g'
    ) AS cleaned
  FROM base
)
SELECT raw, cleaned
FROM clean
""").df()


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,raw,cleaned
0,new 2/2 2 story secondary apt with attached ca...,new 2 2 2 story secondary apt with attached ca...
1,construct new driveway approach of 12 ft\n\n**...,construct new driveway approach of 12 ft no si...
2,master bedroom addition/remodel along with mas...,master bedroom addition remodel along with mas...
3,plumbing for new construction of sfr in the et...,plumbing for new construction of sfr in the et...
4,express: relocate water heater to outside kitc...,express relocate water heater to outside kitch...
5,unit 5101. new 2-story 3 bedroom 3 bathroom to...,unit 5101 new 2 story 3 bedroom 3 bathroom tow...
6,install backflow preventer to swimming pool,install backflow preventer to swimming pool
7,accessory building on pier and beam foundation...,accessory building on pier and beam foundation...
8,irrigation prior to landscape,irrigation prior to landscape
9,total demo of detached shed 245.82 sqft circa ...,total demo of detached shed 245 82 sqft circa ...


In [32]:
import re

# Add head token
phrases["head"] = phrases["phrase"].str.split().str[-1]

# Basic junk filters
def is_junk(p):
    return bool(re.search(r'\b(res|sf|frm|stry|att|ven|watt)\b', p))  # common abbreviations/noise
def head_ok(h):
    return (len(h) >= 4) and (h not in {"existing","create","residence","family","story","single","covered","attached","porch","patio"})

phrases_f = phrases[~phrases["phrase"].apply(is_junk)].copy()

grouped = (phrases_f.groupby("head", as_index=False)
           .agg(phrase_count=("phrase","count"),
                total_freq=("freq","sum"),
                top_phrases=("phrase", lambda s: list(s.head(10)))))

grouped = grouped[grouped["head"].apply(head_ok)].sort_values(["phrase_count","total_freq"], ascending=False)

grouped.head(30)


,head,phrase_count,total_freq,top_phrases
38,garage,4,34530,"[attached garage, wattached garage, car garage..."
57,only,3,9936,"[residence only, service only, srv only]"
81,system,2,11080,"[air system, irrigation system]"
92,with,2,10818,"[system with, residence with]"
64,project,2,8098,"[build project, housing project]"
71,service,2,4676,"[electrical service, electric service]"
12,bedrooms,2,3863,"[3 bedrooms, 4 bedrooms]"
66,remodel,1,16285,[interior remodel]
19,construction,1,8902,[new construction]
40,heater,1,7373,[water heater]


In [33]:
TOP_N = 20  # tune: 15–30 is usually good

candidates = grouped.head(TOP_N).copy()

feature_rules = {
    row["head"]: row["top_phrases"][:6]   # patterns per feature
    for _, row in candidates.iterrows()
}

feature_rules


{'garage': ['attached garage',
  'wattached garage',
  'car garage',
  'detached garage'],
 'only': ['residence only', 'service only', 'srv only'],
 'system': ['air system', 'irrigation system'],
 'with': ['system with', 'residence with'],
 'project': ['build project', 'housing project'],
 'service': ['electrical service', 'electric service'],
 'bedrooms': ['3 bedrooms', '4 bedrooms'],
 'remodel': ['interior remodel'],
 'construction': ['new construction'],
 'heater': ['water heater'],
 'home': ['mobile home'],
 'work': ['duct work'],
 'complete': ['of complete'],
 'heat': ['central heat'],
 'central': ['existing central'],
 'duct': ['without duct'],
 'without': ['or without'],
 'approach': ['driveway approach'],
 'build': ['vol build'],
 'wattached': ['residence wattached']}

In [34]:
import pandas as pd

def mk_cond(patterns):
    # robust OR clause for LIKE
    pats = [p.replace("'", "''") for p in patterns]
    return " OR ".join([f"lower(description) LIKE '%{p}%'" for p in pats])

rows = []
examples = {}

for feat, pats in feature_rules.items():
    cond = mk_cond(pats)
    
    stats = duckdb.query(f"""
        SELECT
          '{feat}' AS feature,
          COUNT(*) AS matches,
          AVG(total_job_valuation_clean) AS avg_val,
          quantile_cont(total_job_valuation_clean, 0.5) AS median_val
        FROM permits_clean
        WHERE description IS NOT NULL
          AND ({cond})
    """).df().iloc[0].to_dict()
    rows.append(stats)
    
    ex = duckdb.query(f"""
        SELECT permit_number, issued_date, original_zip, total_job_valuation_clean, description
        FROM permits_clean
        WHERE description IS NOT NULL
          AND ({cond})
        ORDER BY issued_date DESC NULLS LAST
        LIMIT 8
    """).df()
    examples[feat] = ex

stats_df = pd.DataFrame(rows).sort_values("matches", ascending=False)
stats_df

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

...

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,feature,matches,avg_val,median_val
0,garage,272481,1.623279e+05,110000.0
7,remodel,125587,1.630179e+05,45000.0
2,system,85777,3.674511e+03,1600.0
3,with,84456,2.100913e+05,150000.0
1,only,76629,6.118656e+03,500.0
8,construction,69812,7.983982e+06,385690.0
9,heater,56438,1.175232e+04,600.0
10,home,51196,8.546457e+02,200.0
11,work,50915,3.460436e+04,3980.0
4,project,49897,3.860336e+06,95000.0


## Data cleansing

In [36]:
cols = duckdb.query(f"""
DESCRIBE SELECT * FROM read_csv_auto('{CSV}');
""").df()

cols


,column_name,column_type,null,key,default,extra
0,Permit Type Desc,VARCHAR,YES,None,None,None
1,Permit Num,VARCHAR,YES,None,None,None
2,Permit Class Mapped,VARCHAR,YES,None,None,None
3,Permit Type,VARCHAR,YES,None,None,None
4,Permit Class,VARCHAR,YES,None,None,None
...,...,...,...,...,...,...
63,Applicant Address 2,VARCHAR,YES,None,None,None
64,Applicant City,VARCHAR,YES,None,None,None
65,Applicant Zip,VARCHAR,YES,None,None,None
66,Certificate Of Occupancy,BOOLEAN,YES,None,None,None


In [37]:
cols["column_name"].tolist()


['Permit Type Desc',
 'Permit Num',
 'Permit Class Mapped',
 'Permit Type',
 'Permit Class',
 'Work Class',
 'Condominium',
 'Project Name',
 'Description',
 'TCAD ID',
 'Property Legal Description',
 'Applied Date',
 'Issued Date',
 'Day Issued',
 'Calendar Year Issued',
 'Fiscal Year Issued',
 'Issued In Last 30 Days',
 'Issuance Method',
 'Status Current',
 'Status Date',
 'Expires Date',
 'Completed Date',
 'Total Existing Bldg SQFT',
 'Remodel Repair SQFT',
 'Total New Add SQFT',
 'Total Valuation Remodel',
 'Total Job Valuation',
 'Number Of Floors',
 'Housing Units',
 'Building Valuation',
 'Building Valuation Remodel',
 'Electrical Valuation',
 'Electrical Valuation Remodel',
 'Mechanical Valuation',
 'Mechanical Valuation Remodel',
 'Plumbing Valuation',
 'Plumbing Valuation Remodel',
 'MedGas Valuation',
 'MedGas Valuation Remodel',
 'Original Address 1',
 'Original City',
 'Original State',
 'Original Zip',
 'Council District',
 'Jurisdiction',
 'Link',
 'Project ID',
 'Mast

In [38]:
duckdb.query(f"""
WITH base AS (
  SELECT
    TRY_CAST("Latitude" AS DOUBLE)  AS latitude,
    TRY_CAST("Longitude" AS DOUBLE) AS longitude,
    "Location"                     AS location
  FROM read_csv_auto('{CSV}')
)
SELECT
  COUNT(*) AS total_rows,

  -- Rows with valid lat/lon
  SUM(
    latitude IS NOT NULL
    AND longitude IS NOT NULL
    AND latitude BETWEEN -90 AND 90
    AND longitude BETWEEN -180 AND 180
  )::BIGINT AS rows_with_latlon,

  -- Rows with no lat/lon but Location populated
  SUM(
    (latitude IS NULL OR longitude IS NULL)
    AND location IS NOT NULL
  )::BIGINT AS rows_location_only,

  -- Rows with no geo information at all
  SUM(
    latitude IS NULL
    AND longitude IS NULL
    AND location IS NULL
  )::BIGINT AS rows_no_geo

FROM base;
""").df()


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,total_rows,rows_with_latlon,rows_location_only,rows_no_geo
0,2333734,2301825,0,31909


In [39]:
duckdb.query(f"""
CREATE OR REPLACE VIEW permits_model_ready AS
SELECT
  -- IDs / classification
  "Permit Num"           AS permit_number,
  "Permit Type Desc"     AS permit_type_desc,
  "Permit Type"          AS permit_type,
  "Permit Class"         AS permit_class,
  "Permit Class Mapped"  AS permit_class_mapped,
  "Work Class"           AS work_class,
  "Condominium"          AS condominium,

  -- Text
  "Description"          AS description,

  -- Dates
  TRY_CAST("Issued Date" AS DATE)   AS issued_date,
  TRY_CAST("Applied Date" AS DATE)  AS applied_date,

  -- Status
  "Status Current"       AS status_current,

  -- Geography (Mapbox + modeling)
  TRY_CAST("Latitude" AS DOUBLE)    AS latitude,
  TRY_CAST("Longitude" AS DOUBLE)   AS longitude,
  "Original Zip"         AS original_zip,

  -- Valuation (cleaned)
  CASE
    WHEN TRY_CAST(NULLIF(regexp_replace("Total Job Valuation", '[^0-9.]', '', 'g'), '') AS DOUBLE) IN (0, 1)
      THEN NULL
    ELSE TRY_CAST(NULLIF(regexp_replace("Total Job Valuation", '[^0-9.]', '', 'g'), '') AS DOUBLE)
  END AS total_job_valuation,

  TRY_CAST(NULLIF(regexp_replace("Building Valuation", '[^0-9.]', '', 'g'), '') AS DOUBLE)
    AS building_valuation,
  TRY_CAST(NULLIF(regexp_replace("Electrical Valuation", '[^0-9.]', '', 'g'), '') AS DOUBLE)
    AS electrical_valuation,
  TRY_CAST(NULLIF(regexp_replace("Mechanical Valuation", '[^0-9.]', '', 'g'), '') AS DOUBLE)
    AS mechanical_valuation,
  TRY_CAST(NULLIF(regexp_replace("Plumbing Valuation", '[^0-9.]', '', 'g'), '') AS DOUBLE)
    AS plumbing_valuation,

  -- Physical size
  TRY_CAST(NULLIF(regexp_replace("Total Existing Bldg SQFT", '[^0-9.]', '', 'g'), '') AS DOUBLE)
    AS total_existing_bldg_sqft,
  TRY_CAST(NULLIF(regexp_replace("Total New Add SQFT", '[^0-9.]', '', 'g'), '') AS DOUBLE)
    AS total_new_add_sqft,
  TRY_CAST(NULLIF(regexp_replace("Housing Units", '[^0-9.]', '', 'g'), '') AS DOUBLE)
    AS housing_units,
  TRY_CAST(NULLIF(regexp_replace("Number Of Floors", '[^0-9.]', '', 'g'), '') AS DOUBLE)
    AS number_of_floors,

  -- Optional actor signal
  "Contractor Trade"     AS contractor_trade

FROM read_csv_auto('{CSV}')
WHERE
  TRY_CAST("Latitude" AS DOUBLE) BETWEEN -90 AND 90
  AND TRY_CAST("Longitude" AS DOUBLE) BETWEEN -180 AND 180
""")


In [40]:
duckdb.query("""
SELECT COUNT(*) AS rows
FROM permits_model_ready
""").df()


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,rows
0,2301825


In [41]:
duckdb.query("""
SELECT COUNT(*) AS cols
FROM (DESCRIBE SELECT * FROM permits_model_ready)
""").df()


,cols
0,24


In [42]:
duckdb.query("""
SELECT
  permit_number,
  issued_date,
  permit_type_desc,
  work_class,
  latitude,
  longitude,
  total_job_valuation,
  description
FROM permits_model_ready
USING SAMPLE 10
""").df()


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,permit_number,issued_date,permit_type_desc,work_class,latitude,longitude,total_job_valuation,description
0,2025-005627 MP,2025-04-11,Mechanical Permit,Addition and Remodel,30.291725,-97.718110,NaN,Add a Kitchenette to unit over garage and exte...
1,2024-013909 BP,2024-02-07,Building Permit,Remodel,30.269264,-97.746371,NaN,Finish Out for Level 14 - Demising wall to cre...
2,2024-079266 EP,2024-07-12,Electrical Permit,Homebuilder Loop,30.287359,-97.768960,NaN,Request electrical permit for a temporary Loop...
3,2024-158177 PP,2024-12-10,Plumbing Permit,Repair,30.390765,-97.682845,NaN,Repairing Main Gas Line. Will need gas test fo...
4,2025-098866 EP,2025-08-08,Electrical Permit,Repair,30.224909,-97.873921,NaN,Disconnect/reconnect to tie in pv/ESS componen...
5,2023-164262 PP,2024-05-24,Plumbing Permit,Addition and Remodel,30.254833,-97.714535,NaN,Interior remodel: Converting master bedroom cl...
6,2025-118419 PP,2025-09-21,Plumbing Permit,Irrigation,30.146860,-97.712662,NaN,Irrigation system
7,2021-106166 PP,2021-08-20,Plumbing Permit,New,30.117632,-97.809596,NaN,Estonian 3316 B Right New - 2 Story SFR CONDOM...
8,2025-087240 BP,2025-07-14,Building Permit,Repair,30.315214,-97.655146,NaN,express - replace tub and walls in master bedroom
9,2025-076953 EP,2025-07-22,Electrical Permit,New,30.139695,-97.708066,NaN,*Green Building* New 2-story 5 bedroom 4 bathr...


In [43]:
duckdb.query("""
COPY (SELECT * FROM permits_model_ready)
TO 'permits_model_ready.csv' (HEADER, DELIMITER ',');
""")

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [44]:
gzip permits_model_ready.csv

SyntaxError: invalid syntax (1966268412.py, line 1)

In [45]:
!gzip permits_model_ready.csv


gzip: permits_model_ready.csv: Device or resource busy


In [46]:
!gzip permits_model_ready.csv

In [47]:
duckdb.query("""
CREATE OR REPLACE VIEW permits_model_ultra_clean AS
SELECT
  permit_number,

  permit_type_desc,
  permit_class_mapped,
  work_class,
  condominium,

  description,

  issued_date,
  status_current,

  latitude,
  longitude,
  original_zip,

  total_job_valuation,
  building_valuation,
  electrical_valuation,
  mechanical_valuation,
  plumbing_valuation,

  total_existing_bldg_sqft,
  total_new_add_sqft,
  housing_units,
  number_of_floors,

  contractor_trade

FROM permits_model_ready
""")


In [48]:
duckdb.query("""
SELECT COUNT(*) AS rows
FROM permits_model_ultra_clean
""").df()

duckdb.query("""
DESCRIBE SELECT * FROM permits_model_ultra_clean
""").df()

duckdb.query("""
SELECT *
FROM permits_model_ultra_clean
USING SAMPLE 5
""").df()


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,permit_number,permit_type_desc,permit_class_mapped,work_class,condominium,description,issued_date,status_current,latitude,longitude,...,total_job_valuation,building_valuation,electrical_valuation,mechanical_valuation,plumbing_valuation,total_existing_bldg_sqft,total_new_add_sqft,housing_units,number_of_floors,contractor_trade
0,2025-071048 EP,Electrical Permit,Residential,Upgrade,False,Emergency Replace Damaged Material on undergro...,2025-06-11,Final,30.381145,-97.696629,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,Electrical Contractor
1,2024-107954 MP,Mechanical Permit,Residential,Change Out,<NA>,Replacement of complete existing central heat ...,2024-08-28,Expired,30.246683,-97.829779,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Mechanical Contractor
2,2015-015556 PP,Plumbing Permit,Residential,Repair,False,Austin Water Violation Plumbing Permit,2015-02-18,Final,30.458733,-97.801973,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0,None
3,2025-032316 BP,Building Permit,Residential,Demolition,False,Total Demo of SFR 2500 Sqft Circa 1940,2025-03-21,Final,30.208868,-97.775420,...,NaN,NaN,NaN,NaN,NaN,2500.0,NaN,1.0,1.0,General Contractor
4,2025-000742 PP,Plumbing Permit,Residential,Repair,False,Tunnel Repair on Drain System: Diagnostic reve...,2025-01-08,Final,30.378700,-97.665003,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,Plumbing Contractor


In [49]:
description_clean


NameError: name 'description_clean' is not defined

In [50]:
duckdb.query("""
CREATE OR REPLACE VIEW permits_base_for_modeling AS
SELECT *
FROM permits_model_ultra_clean
""")


In [51]:
duckdb.query("""
CREATE OR REPLACE VIEW permits_with_clean_text AS
SELECT
  *,
  regexp_replace(
    regexp_replace(
      regexp_replace(
        lower(description),
        '[^a-z0-9 ]', ' ', 'g'
      ),
      '\\s+', ' ', 'g'
    ),
    '^ | $', '', 'g'
  ) AS description_clean
FROM permits_base_for_modeling
""")


In [52]:
duckdb.query("""
SELECT
  description,
  description_clean
FROM permits_with_clean_text
USING SAMPLE 10
""").df()


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,description,description_clean
0,New irrigation installation,new irrigation installation
1,Replacing existing subpanel board.,replacing existing subpanel board
2,New construction of multifamily,new construction of multifamily
3,Caprock S405 B Right New - 2 Story SFR w/ 4 be...,caprock s405 b right new 2 story sfr w 4 bedro...
4,New Construction in the ETJ SFR,new construction in the etj sfr
5,Express: Foundation repair,express foundation repair
6,adding a solar pv rooftop system to an existin...,adding a solar pv rooftop system to an existin...
7,Interior Remodel/Garage Conversion - demo of a...,interior remodel garage conversion demo of all...
8,Interior Remodel to Existing Suite 518 (former...,interior remodel to existing suite 518 formerl...
9,Replacement of complete existing central heat ...,replacement of complete existing central heat ...


In [53]:
duckdb.query("""
CREATE OR REPLACE VIEW permits_model_ready_v2 AS
SELECT
  permit_number,

  lower(trim(permit_type_desc))      AS permit_type_desc,
  lower(trim(permit_class_mapped))   AS permit_class_mapped,
  lower(trim(work_class))            AS work_class,
  lower(trim(status_current))        AS status_current,
  lower(trim(contractor_trade))      AS contractor_trade,

  condominium,
  original_zip,

  issued_date,

  latitude,
  longitude,

  total_job_valuation,
  building_valuation,
  electrical_valuation,
  mechanical_valuation,
  plumbing_valuation,

  total_existing_bldg_sqft,
  total_new_add_sqft,
  housing_units,
  number_of_floors,

  description_clean
FROM permits_with_clean_text
""")


In [54]:
duckdb.query("""
SELECT
  COUNT(*) FILTER (WHERE description_clean IS NULL) AS null_descriptions,
  COUNT(*) FILTER (WHERE total_job_valuation IS NULL) AS null_valuation,
  COUNT(*) FILTER (WHERE contractor_trade IS NULL) AS null_trade
FROM permits_model_ready_v2
""").df()


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,null_descriptions,null_valuation,null_trade
0,84,1189108,915499


In [55]:
duckdb.query("""
SELECT COUNT(*) FROM permits_model_ready_v2
""").df()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,count_star()
0,2301825


In [57]:
duckdb.query("""
CREATE OR REPLACE VIEW permits_step6_executed AS
SELECT *
FROM permits_model_ready_v2
WHERE status_current NOT IN (
  'cancelled',
  'canceled',
  'withdrawn',
  'void',
  'voided'
);
""")



In [58]:
SELECT COUNT(*) FROM permits_step6_executed;

SyntaxError: invalid syntax (4262182074.py, line 1)

In [59]:
duckdb.query("SELECT COUNT(*) AS n FROM permits_step6_executed").df()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,n
0,2133476


In [60]:
duckdb.query("""
CREATE OR REPLACE VIEW permits_step7_text AS
SELECT *
FROM permits_step6_executed
WHERE
  description_clean IS NOT NULL
  AND length(description_clean) >= 15
  AND description_clean NOT IN ('permit','service','inspection','test');
""")


In [61]:
duckdb.query("""
SELECT 'step7_text' AS stage, COUNT(*) AS n
FROM permits_step7_text
""").df()


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,stage,n
0,step7_text,1998783


In [77]:
duckdb.query("""
CREATE OR REPLACE VIEW permits_step8_recent AS
SELECT *
FROM permits_step7_text
WHERE issued_date >= DATE '2005-01-01';
""")


In [76]:
duckdb.query("""
SELECT 'step8_recent' AS stage, COUNT(*) AS n
FROM permits_step8_recent
""").df()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,stage,n
0,step8_recent,1201208


In [78]:
duckdb.query("""
CREATE OR REPLACE VIEW permits_with_text_norm AS
WITH base AS (
  SELECT
    *,
    description_clean AS t
  FROM permits_step8_recent
),
norm AS (
  SELECT
    *,
    -- remove common “process” boilerplate words (keep meaning words)
    regexp_replace(t, '\\b(eplan|expedited|review|express)\\b', ' ', 'g') AS t1
  FROM base
),
norm2 AS (
  SELECT
    *,
    -- standardize common abbreviations / tokens (keep these because they are signal)
    regexp_replace(
      regexp_replace(
        regexp_replace(
          regexp_replace(
            regexp_replace(
              regexp_replace(t1,
                '\\bevse\\b', ' ev_charger ', 'g'),
              '\\bpv\\b', ' solar ', 'g'),
            '\\bphotovoltaic\\b', ' solar ', 'g'),
          '\\bsfr\\b', ' single_family ', 'g'),
        '\\badu\\b', ' accessory_dwelling ', 'g'),
      '\\betj\\b', ' extra_territorial_jurisdiction ', 'g'
    ) AS t2
  FROM norm
),
norm3 AS (
  SELECT
    *,
    -- standardize “two story” variants → “2_story”
    regexp_replace(
      regexp_replace(t2, '\\btwo\\s+story\\b', ' 2_story ', 'g'),
      '\\b2\\s+story\\b', ' 2_story ', 'g'
    ) AS t3
  FROM norm2
),
final AS (
  SELECT
    *,
    -- collapse whitespace again and trim
    trim(regexp_replace(t3, '\\s+', ' ', 'g')) AS text_norm
  FROM norm3
)
SELECT * EXCLUDE (t, t1, t2, t3)
FROM final;
""")


In [80]:
duckdb.query("""
SELECT
  description_clean,
  text_norm
FROM permits_with_text_norm
USING SAMPLE 15
""").df()



FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,description_clean,text_norm
0,homebuilder loop austin 2 mile etj austin energy,homebuilder loop austin 2 mile extra_territori...
1,interior alteration of approx 341 sf of existi...,interior alteration of approx 341 sf of existi...
2,home builders loop ref 2024 081332 bp,home builders loop ref 2024 081332 bp
3,new construction of 2 story sfr with 4 beds 3 ...,new construction of 2_story single_family with...
4,express replace 8 windows like for like 2 pati...,replace 8 windows like for like 2 patio doors ...
5,new solar system installation for new resident...,new solar system installation for new resident...
6,interior store front demolition of existing re...,interior store front demolition of existing re...
7,homebuilder loop for new sfr ref 2024 167571 bp,homebuilder loop for new single_family ref 202...
8,repair existing service damage due to ice storm,repair existing service damage due to ice storm
9,install new irrigation system to existing resi...,install new irrigation system to existing resi...
